In [ ]:
import pandas as pd

In [ ]:
artists_df = pd.read_json('data/data_v2/artists.jsonl')

In [ ]:
artists_df.head(50)

In [ ]:
artists_df.shape

In [ ]:
artists_df.isnull().sum()

In [ ]:
sessions_df = pd.read_json('data/data_v2/sessions.jsonl')

In [ ]:
sessions_df.head(50)

In [ ]:
sessions_df.shape

In [ ]:
sessions_df.isnull().sum()

In [ ]:
sessions_df = sessions_df.dropna().astype({"user_id": int})

In [ ]:
sessions_df

In [ ]:
tracks_df = pd.read_json('data/data_v2/tracks.jsonl')

In [ ]:
tracks_df.head(50)

In [ ]:
tracks_df.shape

In [ ]:
tracks_df.isnull().sum()

In [ ]:
users_df = pd.read_json('data/data_v2/users.jsonl')

In [ ]:
users_df.head(50)

In [ ]:
users_df.shape

In [ ]:
users_df.isnull().sum()

In [ ]:
df = pd.merge(sessions_df.mask(sessions_df.eq('None')).dropna(), tracks_df.mask(tracks_df.eq('None')).dropna(), how='inner', left_on = 'track_id', right_on = 'id')

In [ ]:
df.head(50)

In [ ]:
user_track = pd.merge(sessions_df.mask(sessions_df.eq('None')).dropna(), tracks_df.mask(tracks_df.eq('None')).dropna(), how='inner', left_on = 'track_id', right_on = 'id')
user_track = pd.merge(user_track, users_df, how='inner', left_on = 'user_id', right_on = 'user_id')
user_track.drop(["session_id","user_id","id","track_id","id_artist","street"],axis=1,inplace=True)
user_track.rename(columns={"name_x": "track_name", "name_y": "user_name"}, inplace=True)

In [ ]:
user_track